In [1]:
class LIF_Neuron():

    # constant LIF properties
    # baseline charge when neuron is fully neutral
    baseCharge = 4
    # membrane capacitance
    C = 1
    # time interval for our neuron, based on neuron firing 200 times per sec
    dt = 1
    # neuron resistance
    resistance = 16
    # spiking threshold
    threshold = 10
    # additional charge that results from passing threshold
    spike = 1

    # dynamic LIF variables
    inputCharge = 0
    currentCharge = baseCharge

    # Constructor takes in initial input charge
    def __init__(self):
        return

    # Every call using .start() or next() will update the neuron charge
    # Neuron will check if it should spike
    # If it has spiked, we return a 1 and the current charge plus spike
    # otherwise we yield a 0 and the baseline charge
    def run(self, inputCharge):

        # add dV to current charge, dv based on input charge and leak
        self.currentCharge += self.dt * (inputCharge - (self.currentCharge / self.resistance)) / self.C

        # Check if we have reached the threshold
        if self.currentCharge >= self.threshold:
            outputCharge = self.currentCharge
            # resetting charge and spiking
            self.currentCharge = self.baseCharge
            return 1, outputCharge + self.spike

        else:
            return 0, self.baseCharge

In [2]:
from typing import Optional, NamedTuple, Tuple, Any, Sequence
import torch
import torch.nn as nn
import torch.optim as optim
import heapq

In [3]:
import numpy as np

# Neuron 0 and 1 are input neurons. Neurons 2 and 3 are hidden layer neurons. Neuron 4 is the output neuron
# 2 input neurons, 1 for first num and another for second num
# 2 hidden neurons
# 1 output neuron, fires high if XOR, fires low if !XOR

# Creating random initial weights
# Weights represent directed edges in this network:
# 0 (0, 2) input0 to first hidden neuron
# 1 (1, 2) input1 to first hidden neuron
# 2 (0, 3) input0 to second hidden neuron
# 3 (1, 3) input1 to second hidden neuron
# 4 (2, 4) first hidden neuron to output
# 5 (3, 4) second hidden neuron to output
weights = np.random.random(6)
print("Initial random weights: ")
print(weights)

# hardbound for weights
WEIGHT_MAX = 1

# Creating constants for our high and low inputs to represent 0 or 1
LOW_INPUT = 1
HIGH_INPUT = 9

# Constants for our weight change
ALPHA = .175
DECAY = .998

Initial random weights: 
[0.54437436 0.17496952 0.16234648 0.42984071 0.57617136 0.9733045 ]


In [4]:
def add(x,y):
        return x+y

def sub(x,y):
        return x-y

def mul(x,y):
        return x*y

def div(x,y):
        return x/y

F = [add, sub, mul]#, div]

In [5]:
n_operand = 4 #number of operands
n_row = 4 #number of rows
n_column = 2 #number of columns of internal nodes
n_F = len(F) #number of operators

def create_node():
        internal1 = torch.cat((torch.randint(n_F,(n_row,1)),torch.randint(n_operand,(n_row,n_column))),1)
        internal2 = torch.cat((torch.randint(n_F,(n_row,1)),torch.randint(n_row,(n_row,n_column))),1)
        internal = torch.cat((internal1,internal2))
        return internal

In [6]:
def compute(S,operand):
        pheno = S[0]
        i = S[1]
        def compute1(i):
                operator = F[pheno[i][0]]
                output = operator(operand[pheno[i][1]],operand[pheno[i][2]])
                return output
        if i < n_row:
                output = compute1(i)
        else:
                operator = F[pheno[i][0]]
                output = operator(compute1(pheno[i][1]), compute1(pheno[i][2]))
        return output

def mutation(S):
        mu = torch.randn((8,3))#*0.3
        #mu = torch.clamp(mu, -1, 1) + 1
        mutated_internal = torch.round(S[0]+mu)
        l1 = torch.FloatTensor([[0, 0, 0]])
        u1 = torch.FloatTensor([[n_F-1, n_operand-1, n_operand-1]]) #[2,2,2]
        mutated_internal[:4] = torch.max(torch.min(mutated_internal[:4], u1), l1) #clamp the nodes in the first column within the range [l1, u1]
        l2 = torch.FloatTensor([[0, 0, 0]])
        u2 = torch.FloatTensor([[n_F-1, n_row-1, n_row-1]]) #[2,3,3] 
        mutated_internal[4:] = torch.max(torch.min(mutated_internal[4:], u2), l2) #clamp the nodes in the second column within the range [l2, u2]
        mutated_internal = mutated_internal.int()
        mutated_node_index = torch.randint(len(mutated_internal),(1,1)).item()
        mutated_S = [mutated_internal,mutated_node_index]
        return mutated_S

In [7]:
def train(S):

    inputNeuron0 = LIF_Neuron()
    inputNeuron1 = LIF_Neuron()
    middleNeuron2 = LIF_Neuron()
    middleNeuron3 = LIF_Neuron()
    outputNeuron = LIF_Neuron()

    # These values simulate a training current that either force or spike or prohibit a spike
    neuron2training = 0
    neuron3training = 0
    neuron4training = 0
    FORCE_SPIKE = 1000
    PROHIBIT = -2000

    
    N = 1000
    # Train network with this many iterations
    for j in range(N):  #1000

        # print("starting training iteration ", j)
        # print(weights)

        # Each iteration trains every possible input once
        for i in range(4):

            # We find the activity in a 100 time unit interval
            neuron0output = 0
            neuron1output = 0

            neuron0activity = 0
            neuron1activity = 0
            neuron2activity = 0
            neuron3activity = 0
            neuron4activity = 0

            # (0, 0)
            if i == 0:
                neuron0output = inputNeuron0.run(LOW_INPUT)
                neuron1output = inputNeuron1.run(LOW_INPUT)
                neuron2training = PROHIBIT
                neuron3training = FORCE_SPIKE
                neuron4training = PROHIBIT * 2
            # (1, 1)
            elif i == 1:
                neuron0output = inputNeuron0.run(HIGH_INPUT)
                neuron1output = inputNeuron1.run(HIGH_INPUT)
                neuron2training = FORCE_SPIKE
                neuron3training = FORCE_SPIKE
                neuron4training = FORCE_SPIKE
            # (0, 1)
            elif i == 2:
                neuron0output = inputNeuron0.run(LOW_INPUT)
                neuron1output = inputNeuron1.run(HIGH_INPUT)
                neuron2training = FORCE_SPIKE
                neuron3training = FORCE_SPIKE
                neuron4training = PROHIBIT
            # (1, 0)
            elif i == 3:
                neuron0output = inputNeuron0.run(HIGH_INPUT)
                neuron1output = inputNeuron1.run(LOW_INPUT)
                neuron2training = FORCE_SPIKE
                neuron3training = FORCE_SPIKE
                neuron4training = PROHIBIT
            #print(i,neuron0output,neuron1output)
            
            
            for k in range(100):
                # adds 1 if the neuron spiked, adds 0 otherwise
                neuron0activity += neuron0output[0]
                neuron1activity += neuron1output[0]

                # run the or neuron with current given by input neurons and training value
                neuron2charge = neuron0output[1] * weights[0] + neuron1output[1] * weights[1] + neuron2training
                neuron2output = middleNeuron2.run(neuron2charge)
                neuron2activity += neuron2output[0]

                # run the nand neuron with current given by input neurons and training value
                neuron3charge = neuron0output[1] * weights[2] + neuron1output[1] * weights[3] + neuron3training
                neuron3output = middleNeuron3.run(neuron3charge)
                neuron3activity += neuron3output[0]

                # run the output neuron with current given by hidden neurons and training value
                neuron4charge = neuron2output[1] * weights[4] + neuron3output[1] * weights[5] + neuron4training
                neuron4output = outputNeuron.run(neuron4charge)
                neuron4activity += neuron4output[0]

            
            # the activity must be reduced to represent what occurs in one time unit and as a decimal
            neuron0activity = neuron0activity / 1000
            neuron1activity = neuron1activity / 1000
            neuron2activity = neuron2activity / 1000
            neuron3activity = neuron3activity / 1000
            neuron4activity = neuron4activity / 1000

            #print(neuron0activity,neuron1activity,neuron2activity,neuron3activity,neuron4activity)
            # weight adjustments
            # we now calculate weight adjustments based on activity of each neuron and exponential decay

            # print("weight adjustments")

            #weight0dw = ALPHA * neuron0activity * neuron2activity - DECAY * weights[0]
            # print(weight0dw)
            #weights[0] += weight0dw
            #weights[0] = ALPHA * neuron0activity * neuron2activity + DECAY * weights[0]
            weights[0] = compute(S,[1,ALPHA,neuron0activity,neuron2activity]) + DECAY * weights[0]

            #weight1dw = ALPHA * neuron1activity * neuron2activity - DECAY * weights[1]
            # print(weight1dw)
            #weights[1] += weight1dw
            #weights[1] = ALPHA * neuron1activity * neuron2activity + DECAY * weights[1]
            weights[1] = compute(S,[1,ALPHA,neuron1activity,neuron2activity]) + DECAY * weights[1]

            #weight2dw = ALPHA * neuron0activity * neuron3activity - DECAY * weights[2]
            # print(weight2dw)
            #weights[2] += weight2dw
            #weights[2] = ALPHA * neuron0activity * neuron3activity + DECAY * weights[2]
            weights[2] = compute(S,[1,ALPHA,neuron0activity,neuron3activity]) + DECAY * weights[2]

            #weight3dw = ALPHA * neuron1activity * neuron3activity - DECAY * weights[3]
            # print(weight3dw)
            #weights[3] += weight3dw
            weights[3] = compute(S,[1,ALPHA,neuron1activity,neuron3activity]) + DECAY * weights[3]

            #weight4dw = ALPHA * neuron2activity * neuron4activity - DECAY * weights[4]
            # print(weight4dw)
            #weights[4] += weight4dw
            #weights[4] = ALPHA * neuron2activity * neuron4activity + DECAY * weights[4]
            weights[4] = compute(S,[1,ALPHA,neuron2activity,neuron4activity]) + DECAY * weights[4]

            #weight5dw = ALPHA * neuron3activity * neuron4activity - DECAY * weights[5]
            # print(weight5dw)
            #weights[5] += weight5dw
            #weights[5] = ALPHA * neuron3activity * neuron4activity + DECAY * weights[5]
            weights[5] = compute(S,[1,ALPHA,neuron3activity,neuron4activity]) + DECAY * weights[5]

            # Bounding our weights
            for l in range(len(weights)):
                if weights[l] >= WEIGHT_MAX:
                    weights[l] = WEIGHT_MAX
            

In [8]:
# Runs the input 100 times to shown activity in a 100 time unit interval
# If this gate is working, we should see a significant amount of spikes when we expect XOR to return 1
# and far fewer spikes when we expect XOR to return 0
def test(x, y):

    inputNeuron0 = LIF_Neuron()
    inputNeuron1 = LIF_Neuron()
    middleNeuron2 = LIF_Neuron()
    middleNeuron3 = LIF_Neuron()
    outputNeuron = LIF_Neuron()

    totalSpikes = 0

    for i in range(1000):

        neuron0current = 0
        neuron1current = 0

        if x == 0 and y == 0:
            neuron0current = inputNeuron0.run(LOW_INPUT)[1]
            neuron1current = inputNeuron1.run(LOW_INPUT)[1]
        elif x == 0 and y == 1:
            neuron0current = inputNeuron0.run(LOW_INPUT)[1]
            neuron1current = inputNeuron1.run(HIGH_INPUT)[1]
        elif x == 1 and y == 0:
            neuron0current = inputNeuron0.run(HIGH_INPUT)[1]
            neuron1current = inputNeuron1.run(LOW_INPUT)[1]
        elif x == 1 and y == 1:
            neuron0current = inputNeuron0.run(HIGH_INPUT)[1]
            neuron1current = inputNeuron1.run(HIGH_INPUT)[1]

        neuron2output = middleNeuron2.run(neuron0current * weights[0] + neuron1current * weights[1])
        neuron2current = neuron2output[1]

        neuron3output = middleNeuron3.run(14 - (neuron0current * weights[2] + neuron1current * weights[3]))
        neuron3current = neuron3output[1]

        neuron4spikes = outputNeuron.run(neuron2current * weights[4] + neuron3current * weights[5])[0]
        totalSpikes += neuron4spikes
    
    if totalSpikes > 400:
        output = 1
    else:
        output = 0

    # TESTING ONLY REMOVE LATER
    return totalSpikes, output

In [9]:
def accuracy():
    prediction = [test(0, 0)[1],test(0, 1)[1],test(1, 0)[1],test(1, 1)[1]]
    label = [0,1,1,0]
    return sum(abs(np.array(prediction)-np.array(label)))    

In [12]:
#opernad = [1,alpha,pre,post]
internal1 = torch.IntTensor([[2,0,1],[2,2,3],[1,1,1],[0,0,0],[2,0,1],[0,0,0],[0,0,0],[0,0,0]])
node_index1 = torch.randint(len(internal1),(1,1)).item()
#node_index1 = 4
S1 = [internal1,node_index1]
internal2 = create_node()
node_index2 = torch.randint(len(internal2),(1,1)).item()
S2 = [internal2,node_index2]
Stab = [S1,S2]
train(S1)
L1 = accuracy()
weights = np.random.random(6)
train(S2)
L2 = accuracy()
weights = np.random.random(6)
Ltab = [L1,L2]
print(Ltab)

[2, 2]


In [13]:
gen = 501
for g in range(gen):
    Smutab = []
    Smu11 = mutation(Stab[0])
    Smu12 = mutation(Stab[0])
    Smu21 = mutation(Stab[1])
    Smu22 = mutation(Stab[1])
    Smutab.append(Smu11)
    Smutab.append(Smu12)
    Smutab.append(Smu21)
    Smutab.append(Smu22)
    #print(Smutab)
    Lmutab = []
    train(Smu11)
    weights = np.random.random(6)
    Lmutab.append(accuracy())
    train(Smu12)
    weights = np.random.random(6)
    Lmutab.append(accuracy())
    train(Smu21)
    weights = np.random.random(6)
    Lmutab.append(accuracy())
    train(Smu22)
    weights = np.random.random(6)
    Lmutab.append(accuracy())
    #print(Lmutab)
    Ljointtab = Ltab + Lmutab
    Sjointtab = Stab + Smutab
    good_index = heapq.nsmallest(2,range(len(Ljointtab)), Ljointtab.__getitem__)
    Stab = [Sjointtab[m] for m in good_index]
    Ltab = [Ljointtab[n] for n in good_index]
    if g % 10 == 0:
        print(Ljointtab)
        print(Ltab)
 

[2, 2, 2, 2, 2, 2]
[2, 2]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 3, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 1, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 3, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2, 2, 2]
[1, 1]
[1, 1, 2, 2,

In [14]:
print(Stab)

[[tensor([[2, 0, 0],
        [2, 2, 2],
        [2, 3, 1],
        [0, 0, 0],
        [2, 0, 2],
        [0, 0, 1],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32), 7], [tensor([[2, 3, 3],
        [2, 1, 2],
        [1, 1, 0],
        [1, 0, 0],
        [0, 0, 2],
        [1, 0, 3],
        [1, 0, 3],
        [0, 1, 3]], dtype=torch.int32), 1]]


## TEST===============================

In [11]:
train(S1)
print("Trained weights")
print(weights)

print("\nFinding spikes rates in a 100 time unit interval")
print("Expected results for a working xor network is significant spike rate difference between 0 and 1 outputs.")
print("(0, 0): " + str(test(0, 0)))
print("(0, 1): " + str(test(0, 1)))
print("(1, 0): " + str(test(1, 0)))
print("(1, 1): " + str(test(1, 1)))

Trained weights
[0.40284207 0.40272383 0.43791171 0.43766393 0.1451231  0.17289226]

Finding spikes rates in a 100 time unit interval
Expected results for a working xor network is significant spike rate difference between 0 and 1 outputs.
(0, 0): (363, 0)
(0, 1): (500, 1)
(1, 0): (500, 1)
(1, 1): (375, 0)


In [21]:
prediction = [0, 0, 0, 0]
label = [0, 1, 1, 0]
a = abs(np.array(prediction)-np.array(label))
sum(a)

2